# Pluggy 使用示例（Jupyter 笔记本）

本笔记本演示如何使用 pluggy（hookspec + hookimpl + PluginManager），并同时包含基础 Python、数值计算、绘图、单元测试与在 VS Code 中运行/调试的示例。打开后可以逐单元运行（推荐在项目虚拟环境中运行）。

要点：
- HookSpec：定义 hook 接口（使用 @pluggy.HookspecMarker）
- HookImpl：插件实现（使用 @pluggy.HookimplMarker）
- PluginManager：注册 hook 规范与插件，调用 pm.hook.<name>。

下面各章节会逐步展示环境、基础语法、测试、绘图与 pluggy 示例。

## 1. 导入与环境设置

本节演示如何安装/导入本笔记本需要的库，并说明在 VS Code 中选择 Python kernel 的方法。建议使用项目的虚拟环境（例如 `.venv`）。

下面的代码单元会尝试使用 `%pip` 安装必要依赖（只在需要时安装）。如果你在受限环境中，请在终端手动安装。

In [ ]:
# 安装依赖（可在笔记本中直接运行）
# 建议在虚拟环境中运行；如果已安装这些包，这条命令会跳过
%pip install -q pluggy numpy pandas matplotlib sympy pytest

import sys
import pluggy
print('Python:', sys.version.splitlines()[0])
print('Pluggy:', pluggy.__version__)

## 2. Python 基本语法示例（变量、列表、循环、条件）

下面给出若干简短示例，便于在输出窗格中查看结果。

In [ ]:
# 变量、列表、循环、条件
x = 10
squares = [i*i for i in range(5)]
print('x =', x)
print('squares =', squares)

for i in range(3):
    if i % 2 == 0:
        print(i, 'even')
    else:
        print(i, 'odd')

# 捕获常见错误示例
try:
    a = 1 / 0
except ZeroDivisionError as exc:
    print('捕获到错误：', exc)

## 3. 函数与单元测试（定义、注释、pytest）

示例定义函数并使用简单断言作为测试示例。也会说明如何在 VS Code 中运行 pytest。

In [ ]:
# 定义函数（阶乘）并通过断言测试

def factorial(n: int) -> int:
    """计算 n 的阶乘（仅接受非负整数）"""
    if n < 0:
        raise ValueError('n must be >= 0')
    res = 1
    for i in range(2, n+1):
        res *= i
    return res

# 简单断言测试（在 notebook 中演示）
assert factorial(0) == 1
assert factorial(5) == 120
print('factorial 测试通过')

# 在项目中使用 pytest：把测试放在 tests/ 目录，运行 `pytest -q` 或在 VS Code 的测试面板运行。

## 4. 数值计算与绘图（NumPy、Matplotlib）

生成带噪声的线性数据，拟合并绘制散点图与拟合直线。

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# 生成数据
rng = np.random.default_rng(42)
x = np.linspace(0, 10, 50)
a, b = 2.5, 1.0
noise = rng.normal(scale=2.0, size=x.shape)
y = a * x + b + noise

# 简单线性拟合（最小二乘）
coeffs = np.polyfit(x, y, 1)
poly = np.poly1d(coeffs)

plt.scatter(x, y, label='data')
plt.plot(x, poly(x), color='red', label=f'fit: y={coeffs[0]:.2f}x+{coeffs[1]:.2f}')
plt.legend()
plt.title('线性拟合示例')
plt.show()

## 5. 数据处理示例（Pandas：读取 CSV、清洗、分组聚合）

使用内置的 CSV 示例，展示读取、处理缺失值、按列分组并聚合。

In [ ]:
import pandas as pd
from io import StringIO

csv = StringIO('''category,value
A,10
B,20
A,15
B,
C,5
''')
df = pd.read_csv(csv)
print('原始数据:')
print(df)

# 清洗：填充缺失值
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['value'] = df['value'].fillna(0)
print('\n清洗后:')
print(df)

print('\n按 category 聚合:')
print(df.groupby('category', as_index=False).agg({'value':'sum'}))

## 6. 符号计算与数学展示（SymPy）

演示符号微积分并在笔记本中以 LaTeX 的形式显示结果，例如计算 $$\int_0^1 x^2\,dx=\tfrac{1}{3}$$。

In [ ]:
import sympy as sp
from IPython.display import display, Math

x = sp.symbols('x')
expr = x**2
res = sp.integrate(expr, (x, 0, 1))
print('积分结果（SymPy 对象）:', res)
# 以 LaTeX 显示
display(Math(r"\int_0^1 x^2\,dx=\frac{1}{3} = %s" % sp.latex(res)))

## 7. 在 VS Code 中运行与调试

- 在 VS Code 中选择解释器（右下角或命令面板：Python: Select Interpreter），选择项目虚拟环境。
- 逐单元运行：点击每个单元左侧的运行按钮或使用上方运行命令。
- 在集成终端运行：`python examples/pluggy_example.py`。
- 调试：可以把关键函数提取到 .py 文件，设置断点并使用 Run > Start Debugging（或在调试面板）。

笔记本中也可使用 `%debug` 在异常后进入交互调试（仅限 IPython）。

## 8. 导出与版本控制

- 导出为 Python 脚本：在命令面板选择 "Export As" 或使用 `jupyter nbconvert --to script examples/pluggy_example.ipynb`。
- 导出为 HTML：`jupyter nbconvert --to html examples/pluggy_example.ipynb`。
- 将生成文件加入 git：

```bash
git add examples/pluggy_example.ipynb
git add examples/pluggy_example.py  # 如果导出为脚本
git commit -m "Add pluggy example notebook"
git push
```

建议添加 `.gitignore` 忽略 large outputs 和虚拟环境（例如 `.venv/`）。

---

## Pluggy 专门示例（演示 HookSpec / HookImpl / PluginManager）

下面是最小可运行的 pluggy 示例：定义 hookspec、两个插件实现，注册并调用 hook。

In [ ]:
import pluggy

HOOK_NAMESPACE = 'astrbot_canary'
pm = pluggy.PluginManager(HOOK_NAMESPACE)

# 定义 hookspec
class Spec:
    @pluggy.HookspecMarker(HOOK_NAMESPACE)
    def greet(self, name: str) -> str:
        """返回一段问候语"""

# 插件实现
class PluginA:
    @pluggy.HookimplMarker(HOOK_NAMESPACE)
    def greet(self, name: str) -> str:
        return f'PluginA: Hello, {name}!'

class PluginB:
    @pluggy.HookimplMarker(HOOK_NAMESPACE)
    def greet(self, name: str) -> str:
        return f'PluginB: Hi there, {name}!'

# 注册并调用
pm.add_hookspecs(Spec)
pm.register(PluginA())
pm.register(PluginB())

print('pluggy version:', pluggy.__version__)
print('\ngreet results:')
for r in pm.hook.greet(name='Alice'):
    print(' ', r)

# 取第一个非 None 返回值（兼容各种 pluggy 版本）
results_bob = pm.hook.greet(name='Bob')
first = next((r for r in results_bob if r is not None), None)
print('\nfirst_result:', first)